In [2]:
import boto3

ec2_client = boto3.client('ec2')
ec2_resource = boto3.resource('ec2')
rds_client = boto3.client('rds')
efs_client = boto3.client('efs')
elbv2_client = boto3.client('elbv2')

In [13]:
# Create VPC with CIDR Block 10.0.0.0/16
vpc = ec2_resource.create_vpc(CidrBlock = '10.0.0.0/16')
vpc.wait_until_available()
vpc.create_tags(Tags=[{'Key': 'Name', 'Value': 'wordpress-serverless-vpc'}])

[ec2.Tag(resource_id='vpc-00bc214477e067ad4', key='Name', value='wordpress-serverless-vpc')]

In [ ]:
ec2_client = boto3.client('ec2', region_name='us-west-2')  # Adjust region_name as needed
response = ec2_client.describe_availability_zones()
available_zones = [zone['ZoneName'] for zone in response['AvailabilityZones']]
print("Available AZs:", available_zones)

In [15]:
# Create public and private subnets of 10.0.x.0/24
public_subnet_a = vpc.create_subnet(CidrBlock='10.0.1.0/24', AvailabilityZone='us-west-2a')
public_subnet_a.create_tags(Tags=[{'Key': 'Name', 'Value': 'public-subnet-A'}])

public_subnet_b = vpc.create_subnet(CidrBlock='10.0.2.0/24', AvailabilityZone='us-west-2b')
public_subnet_b.create_tags(Tags=[{'Key': 'Name', 'Value': 'public-subnet-B'}])

private_subnet_a = vpc.create_subnet(CidrBlock='10.0.3.0/24', AvailabilityZone='us-west-2a')
private_subnet_a.create_tags(Tags=[{'Key': 'Name', 'Value': 'private-subnet-A'}])

private_subnet_b = vpc.create_subnet(CidrBlock='10.0.4.0/24', AvailabilityZone='us-west-2b')
private_subnet_b.create_tags(Tags=[{'Key': 'Name', 'Value': 'private-subnet-B'}])

[ec2.Tag(resource_id='subnet-0a26b8ce072a083ce', key='Name', value='private-subnet-B')]

In [17]:
# Create Internet Gateway and Attach it to VPC
igw = ec2_resource.create_internet_gateway()
vpc.attach_internet_gateway(InternetGatewayId = igw.id)
igw.create_tags(Tags=[{'Key': 'Name', 'Value': 'wordpress-serverless-igw'}])

[ec2.Tag(resource_id='igw-0b175b4f33b677625', key='Name', value='wordpress-serverless-igw')]

In [20]:
# Create route tables for public subnets
public_route_table = vpc.create_route_table()
public_route_table.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId = igw.id)
public_route_table.associate_with_subnet(SubnetId = public_subnet_a.id)
public_route_table.associate_with_subnet(SubnetId = public_subnet_b.id)
public_route_table.create_tags(Tags=[{'Key': 'Name', 'Value': 'wordpress-public-route-table'}])

[ec2.Tag(resource_id='rtb-09987fcf8bcb6228f', key='Name', value='wordpress-public-route-table')]

In [25]:
# Allocate Elastic IPs for NAT Gateway
eip_a = ec2_client.allocate_address(Domain='vpc')
eip_a_id = eip_a['AllocationId']

# Name the Elastic IP
ec2_client.create_tags(Resources=[eip_a['AllocationId']], Tags=[{'Key': 'Name', 'Value': 'wordpress-eip-nat-gateway-a'}])

# Create NAT Gateway in the public subnet A and tag it
nat_gateway_a = ec2_client.create_nat_gateway(SubnetId=public_subnet_a.id, AllocationId=eip_a_id)
nat_gateway_a_id = nat_gateway_a['NatGateway']['NatGatewayId']

# Wait for the NAT Gateway to be available before tagging
ec2_client.get_waiter('nat_gateway_available').wait(NatGatewayIds=[nat_gateway_a_id])
ec2_client.create_tags(Resources=[nat_gateway_a_id], Tags=[{'Key': 'Name', 'Value': 'wordpress-nat-gateway-a'}])

# Repeat for NAT Gateway B
eip_b = ec2_client.allocate_address(Domain='vpc')
eip_b_id = eip_b['AllocationId']

# Name the Elastic IP
ec2_client.create_tags(Resources=[eip_b['AllocationId']], Tags=[{'Key': 'Name', 'Value': 'wordpress-eip-nat-gateway-b'}])

nat_gateway_b = ec2_client.create_nat_gateway(SubnetId=public_subnet_b.id, AllocationId=eip_b_id)
nat_gateway_b_id = nat_gateway_b['NatGateway']['NatGatewayId']

# Wait for the NAT Gateway B to be available and tag it
ec2_client.get_waiter('nat_gateway_available').wait(NatGatewayIds=[nat_gateway_b_id])
ec2_client.create_tags(Resources=[nat_gateway_b_id], Tags=[{'Key': 'Name', 'Value': 'wordpress-nat-gateway-b'}])


{'ResponseMetadata': {'RequestId': '13059ed4-51ec-48fb-b470-f53165788f1a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '13059ed4-51ec-48fb-b470-f53165788f1a',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '221',
   'date': 'Mon, 26 Feb 2024 20:15:43 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [26]:
# Create Route Tables for Private Subnets
private_route_table_a = vpc.create_route_table()
private_route_table_a.create_route(DestinationCidrBlock='0.0.0.0/0', NatGatewayId=nat_gateway_a['NatGateway']['NatGatewayId'])
private_route_table_a.associate_with_subnet(SubnetId=private_subnet_a.id)
private_route_table_a.create_tags(Tags=[{'Key': 'Name', 'Value': 'wordpress-private-route-table-a'}])

private_route_table_b = vpc.create_route_table()
private_route_table_b.create_route(DestinationCidrBlock='0.0.0.0/0', NatGatewayId=nat_gateway_b['NatGateway']['NatGatewayId'])
private_route_table_b.associate_with_subnet(SubnetId=private_subnet_b.id)  # Corrected this line
private_route_table_b.create_tags(Tags=[{'Key': 'Name', 'Value': 'wordpress-private-route-table-b'}])


[ec2.Tag(resource_id='rtb-0b9b2e6cb823f790e', key='Name', value='wordpress-private-route-table-b')]

In [28]:
alb_security_group = ec2_client.create_security_group(
    GroupName='ALB_group',
    Description='Security Group for Application Load Balancer Allowing Web Traffic',
    VpcId=vpc.id  # If `vpc` is a VPC resource object, use `vpc.vpc_id` instead
)

ec2_client.authorize_security_group_ingress(
    GroupId=alb_security_group['GroupId'],
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 80,
            'ToPort': 80,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        },
        {
            'IpProtocol': 'tcp',
            'FromPort': 443,
            'ToPort': 443,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)


{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0205d75ec77944894',
   'GroupId': 'sg-092063b2410bfcb9a',
   'GroupOwnerId': '058264094199',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '0.0.0.0/0'},
  {'SecurityGroupRuleId': 'sgr-03a78486455ae8f7e',
   'GroupId': 'sg-092063b2410bfcb9a',
   'GroupOwnerId': '058264094199',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 443,
   'ToPort': 443,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': 'a996f1c8-008e-4a0e-bce6-ca8675b06fed',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a996f1c8-008e-4a0e-bce6-ca8675b06fed',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1126',
   'date': 'Mon, 26 Feb 2024 20:26:55 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [29]:
# Create RDS Security Group
rds_security_group = ec2_client.create_security_group(
    GroupName='RDS_group',
    Description='Security Group for RDS MySQL Access',
    VpcId=vpc.id
)

# Authorize MySQL traffic on port 3306 from within the VPC
ec2_client.authorize_security_group_ingress(
    GroupId=rds_security_group['GroupId'],
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 3306,
            'ToPort': 3306,
            'IpRanges': [{'CidrIp': '10.0.0.0/16'}]  # Adjust the CIDR to match your VPC's CIDR block
        }
    ]
)


{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0970c981d94e6b069',
   'GroupId': 'sg-01523779b35d49453',
   'GroupOwnerId': '058264094199',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 3306,
   'ToPort': 3306,
   'CidrIpv4': '10.0.0.0/16'}],
 'ResponseMetadata': {'RequestId': '660591d2-4f02-407a-9577-aedb84cd01b9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '660591d2-4f02-407a-9577-aedb84cd01b9',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '725',
   'date': 'Mon, 26 Feb 2024 20:27:05 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [30]:
# Create EFS Security Group
efs_security_group = ec2_client.create_security_group(
    GroupName='EFS_group',
    Description='Security Group for EFS Access',
    VpcId=vpc.id
)

# Authorize NFS traffic on port 2049 from within the VPC
ec2_client.authorize_security_group_ingress(
    GroupId=efs_security_group['GroupId'],
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 2049,
            'ToPort': 2049,
            'IpRanges': [{'CidrIp': '10.0.0.0/16'}]  # Adjust the CIDR to match your VPC's CIDR block
        }
    ]
)


{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0ec2b3f83e88c0f3f',
   'GroupId': 'sg-0ac432ccea836a36d',
   'GroupOwnerId': '058264094199',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 2049,
   'ToPort': 2049,
   'CidrIpv4': '10.0.0.0/16'}],
 'ResponseMetadata': {'RequestId': 'b68a34e9-379a-497e-86ee-02b2176fcf98',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b68a34e9-379a-497e-86ee-02b2176fcf98',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '725',
   'date': 'Mon, 26 Feb 2024 20:27:13 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}